<h1 class="western" style="font-weight: normal; text-align: center;"><span style="color: #0000ff;"><span style="font-family: Liberation Serif, serif;">Chemical Evolution of the Milky Way</span></span></h1>

###### Authors:

* .
* .
* .
* .


In [1]:
# Libraries 

import numpy as np
import matplotlib.pylab as plt

## 1 - Description




<h3>1.1 Motivation:</h3>
<p>This code is a product of the project "Chemical Evolution of MW" developed at the 41th ISYA, Socorro -&nbsp; Colombia by the authors above. <span lang="en">Our goal is to perform a description of the distribution of metallicity of the stars of our galaxies over time, focusing mainly on the distribution observed in the current time. In order to do that, we built a code which solves the system of differential equations developed by xxxxx, called Closed Box Model. <br /></span></p>


### 1.2 Goals:

* Construct a code that returns the **probability distribution of metallicity at different times** and
make a **comparison with the literature**.
* Compare the model with **observational data** in order to identify the efficiency of the treatment.
* Consider a **modification of the model**, assuming mass outflows and identify what the final outcome change is.

WHAT ELSE??

### 1.3 Closed Box Model

For a complete understanding of the evolution of the chemical elements in our Galaxy, we need to identify which are the chemical processes that influence the appearance of these elements and which are the mechanisms that carry out their distribution.

The so-called light chemical elements, such as H, He and D, were formed in the primordial nucleosynthesis at the beginning of the Universe's history and are the most abundant elements still today. Elements heavier than helium are called metals and are formed inside the stars. The evolution of these objects leads to their death and, in the case of more massive stars, the enrichment of the interstellar medium through the process of supernova type II explosion. Besides this process, the metals can be formed during the explosion of type Ia supernovae (Giant Red system + white ana). It is important to note that the fraction of formed metals varies for each type of supernova.

Considering such information, our treatment is based on the closed box model, initially proposed by Txxx (1980). This model assumes:

* the galaxy's gas is well-mixed (had the same initial com position everywhere)
* IRA (instantaneous reciclying assumption) -the (high-mass) stars return their nucleosynthetic products rapidly, much faster than the time to form a significant fraction of the stars. 
* no gas escapes from or is added to the galaxy 

We get the following distribution:

$$
Z = y ln \frac{1}{\mu}
$$


## 2 - Initial conditions:
we need reliable references


In [2]:
M0_gas   =4e10  #en Msun
gal_diam =30000 #en pc (30 en kpc)
gal_radi =15000 #en pc (15 en kpc)
gal_width=400   #en pc (0.4 en kpc)
gal_vol  =(np.pi*gal_radi**2)*gal_width
volum_gas_density=1.*M0_gas/gal_vol
surf_gas_density =volum_gas_density*gal_width
#eff=2.5*10^(-4)

## 3 - Functions:

We define some functions used in our methodology:



### 3.1 - IMF: Initial Mass Function

The initial mass function (IMF) is an empirical function that describes the initial distribution of masses for a population of stars. The IMF is often given as a probability distribution function (PDF) for the mass at which a star enters the main sequence.

<p><img src="http://www.astro.ljmu.ac.uk/~ikb/research/IMFs.gif" alt="" width="341" height="310" /></p>

Among the different models of MFIs presented, we adopted the model of Salpeter (year) for our model. **The IMF is constante in time.**

$$
IMF = \frac{1}{16.5816} * m^{-2.35}
$$

[IMF] = adm

In [3]:
def imf_salp(m):
    imf = (1./ 16.5816)*(m**(-2.35))
    return imf

### 3.2 - Star Formation Rate

The star formation rate is the total mass of stars formed per year. 

**Kennicutt-Schmidt Law**

A key ingredient in the understanding and modelling of galaxy evolution is the relationship between the large-scale star formation rate (SFR) and the physical conditions in the interstellar medium (ISM). The KS law relates the star formation rate of a galaxy with its density surface of gas:


$$
\psi(t) = \nu \sigma_{gas}^{k} =SFR
$$

Each:

$[\psi(t)] =  \frac{M_{\odot}}{yr \cdot Mpc^{2}}$

$[\sigma_{gas}] =  \frac{M_{\odot}}{\cdot Mpc^{2}}$

$k = 1.4$ y $\nu = 2.5*10^{-4} M_{solar} /yr / kpc^2$ 


In [4]:
##----------STAR FORMATION RATE----------------
def sfr(dens,eff=176714586.76442587,exp=1.4):
 '''
 eff=1Gyr^-1 para el disco
 exp=1.5
 '''
 return eff*(dens)**exp


### 3.4 - Remnant mass


In [6]:
def remnant_mass(m):
    '''
    Calculates the remnant mass of a star of mass m.

    The formulism is based on Ferreras & Silk 2000 (ApJ 532 193)
    which in turn is based on Iben & Tsutukov 1984, Woosley & Weaver 1995.
    This accounts for stars with mass above 40Msun not returning all their material into the ISM
    '''
    if m <= 9.0:
        rem_mass = 0.106*m + 0.446
    elif (m > 9.0) & (m < 25.0):
        rem_mass = 1.5
    else:
        rem_mass = 0.61*m - 13.75

    rem_mass = rem_mass#*u.solMass
    return rem_mass

### 3.5 - Gas Ejected

This function evaluates the second argument of the equation (x) which relates with the amount of material ejected but a dying star. 

It assumes the instantaneous recycling approximation: if $ m < 1 M_{\odot}$, there is no contribution. Stars which mass is $m > 1 M_{\odot}$ dies instantaneously.

So, for the contribuion stars:

$$
E(m) = \int_{m(t)}^{\inf} (m-M_r) \psi(t-\tau_r) \phi(m) dm
$$

m is the initial mass of the star,
Mr is the mass of the remnant star **(m - Mr) -> Mass ejected by the star at death **

**$\psi(t)$: star formation rate.** The $\tau_R$ is neglected due to IRA.

**$\phi(m)$: inicial mass function**

**Obs.:** This function calls the functions remnant_mass(m) and sfc(des)


In [7]:
##################################
def eject_gas(m,sfc_dens):
 #age=stellar_age(m)
 if m<1:
    return 0
 else:
    eject=(m-remnant_mass(m))*sfr(sfc_dens,eff=176714586.76442587,exp=1.4)*imf_salp(m)
    return eject
##################################

### 3.6 - Metallicity evolution


Assuming IRA, we get the relationship between the metallicity and the mass of the gas in a certain time:

$$
Z(t) = y + ln\left(\frac{M_{TOT}}{M_{gas}(t)}\right) 
$$

y = effective yield
We assume:

$y = 0.5 Z_{\odot} = 0.01$


In [8]:
def Z_evol(M_gas,M_tot,Zyield=0.01):
    '''
    entrega la masa de los metales respecto a la masa de gas
    '''
    return Zyield*np.log(1.*M_tot/M_gas)
######################################################################################

## 4 - Methodology:


We will follow the discussion in Matteucci (2016) and chapter 3 of the book "Chemical Evolution blablaba". As commented, the Simple Model of chemical evolution assumes that the system is evolving as a closed-box,
without inflows or outflows, the IMF is constant in time, the chemical composition of the gas
is primordial and the mixing between the chemical products ejected by stars and the ISM is
instantaneous, plus IRA.



### 4.1 - 

In [9]:
# Gas mass in a given step time:
m_gas_array=[]
# Gas density volume in a given step time
volum_gas_density_array=[]
# Gas density surface in a given step time
surf_gas_density_array =[]
# Time
time=[]
# 
eg_array=[]
#
sg_array=[]
# Metallicity over time:
Z_array=[]

#First mass of the gas is the total mas
m_gas_array.append(M0_gas)

#TVOlume density of the Galaxy (disc approximation):
volum_gas_density_array.append(gal_vol)

#Surface density of the Galaxy (disc approximation):

surf_gas_density_array.append(surf_gas_density)
eg_array.append(0)
sg_array.append(0)
Z_array.append(0)

#Z_array.append(0)


Numeric method:

$$
\frac{dM_{gas}}{dt} = -\psi(t) + \int_{m(t)}^{\inf} (m-M_r) \psi(t-\tau_r) \phi(m) dm
$$

$$
\frac{M_{i+1} - M_{i}}{\Delta t} = -\psi(t) + \int_{m(t)}^{\inf} (m-M_r) \psi(t-\tau_r) \phi(m) dm
$$

$\Delta t << 1$, so, for each step:

$$M_{i+1} = M_{i} - \phi(t_{i}) \Delta t - \Delta t \int_{m(t)}^{\inf} (m-M_r) \psi(t-\tau_r) \phi(m) dm
$$ ???
$$
\frac{M_{new} - M_{old}}{\Delta t} = K
$$

so is going to be an iteration for calculating new mases thinking in an update

$$
M_{new} = K* \Delta t -M_{old} = -\psi(t)* \Delta t- M_{old} = \nu \sigma_{gas} (m_{old}) ^{k} * \Delta t - M_{old}
$$
1. Integration of $E = \Delta t \int_{m(t)}^{\inf} (m-M_r) \psi(t-\tau_r) \phi(m) dm$ 
2. Evaluation of the Star Formation Rate
3. Evaluation of $ ( - \phi + E)$ : Variation of the mass gas in a time step
4. Product of $\phi$ and $E$ by the time step, as the numeric method sugests. 
5. Update of the mass: right side of the last equation
6. Update of the density volume: current mass of the gas over the total volume of the Galaxy (fixed number).
7. Update of the surface density: current volume density multiplied by the width of the galaxy. 
8. Evaluation of the metallicity distribution




In [10]:
from scipy import integrate
i = 0
dt=0.001
steps = 10000
while i < steps:
    print (i,surf_gas_density_array[-1])
    t=i*dt
    time.append(t)
    # Calculating the E for a giving time step
    eg=integrate.romberg(eject_gas,0.1,100,args=(surf_gas_density_array[-1],),divmax=30,rtol=1e10) #1
    sg=sfr(surf_gas_density_array[-1],eff=176714586.76442587,exp=1.4) #2
    esg=eg-sg #3 
    eg_array.append(eg*dt) # 4.1
    sg_array.append(sg*dt) # 4.2
    current_Mgas   =m_gas_array[-1]+ esg*dt # 5
    current_Vdensity=1.*current_Mgas/gal_vol #6
    current_Adensity=current_Vdensity*gal_width #7

    
    Z_array.append(Z_evol(M_gas=current_Mgas,M_tot=M0_gas,Zyield=0.01))
    m_gas_array.append(current_Mgas)
    volum_gas_density_array.append(current_Vdensity)
    surf_gas_density_array.append(current_Adensity)
    i+=1
time.append(10)

0 56.58842421045167
1 56.51838144163498
2 56.44846001715054
3 56.37865966679622
4 56.30898012110504
5 56.23942111134278
6 56.169982369505625
7 56.10066362831782
8 56.031464621229354
9 55.96238508241359
10 55.893424746765
11 55.82458334989677
12 55.75586062813864
13 55.68725631853444
14 55.61877015883992
15 55.550401887520465
16 55.48215124374876
17 55.414017967402586
18 55.34600179906256
19 55.27810248000989
20 55.21031975222412
21 55.14265335838095
22 55.075103041849935
23 55.00766854669238
24 54.940349617659024
25 54.87314600018798
26 54.8060574404024
27 54.73908368510841
28 54.67222448179289
29 54.605479578621306
30 54.538848724435596
31 54.472331668751984
32 54.405928161758865
33 54.33963795431465
34 54.27346079794565
35 54.207396444843994
36 54.14144464786548
37 54.07560516052751
38 54.00987773700693
39 53.94426213213804
40 53.878758101410455
41 53.81336540096705
42 53.748083787601885
43 53.68291301875819
44 53.61785285252628
45 53.55290304764153
46 53.48806336348233
47 53.4233335

968 21.25040597306624
969 21.23262896011934
970 21.21487276357614
971 21.197137352103187
972 21.179422694424662
973 21.161728759322248
974 21.144055515635014
975 21.12640293225927
976 21.108770978148492
977 21.09115962231313
978 21.073568833820538
979 21.055998581794828
980 21.038448835416748
981 21.020919563923567
982 21.003410736608934
983 20.98592232282279
984 20.968454291971213
985 20.951006613516302
986 20.933579256976074
987 20.916172191924336
988 20.898785387990547
989 20.88141881485971
990 20.86407244227227
991 20.846746240023954
992 20.8294401779657
993 20.812154226003486
994 20.794888354098255
995 20.777642532265773
996 20.76041673057652
997 20.743210919155565
998 20.72602506818246
999 20.708859147891104
1000 20.69171312856964
1001 20.674586980560335
1002 20.65748067425947
1003 20.640394180117212
1004 20.6233274686375
1005 20.606280510377932
1006 20.589253275949666
1007 20.572245736017273
1008 20.55525786129864
1009 20.53828962256486
1010 20.52134099064011
1011 20.50441193640

1967 10.330185653925488
1968 10.323709814935734
1969 10.317239658682661
1970 10.310775178755694
1971 10.3043163687531
1972 10.297863222281954
1973 10.291415732958157
1974 10.284973894406397
1975 10.278537700260141
1976 10.272107144161623
1977 10.265682219761826
1978 10.259262920720472
1979 10.252849240706006
1980 10.246441173395578
1981 10.240038712475037
1982 10.233641851638911
1983 10.227250584590394
1984 10.220864905041328
1985 10.214484806712202
1986 10.20811028333212
1987 10.201741328638802
1988 10.19537793637856
1989 10.18902010030629
1990 10.18266781418546
1991 10.176321071788086
1992 10.169979866894725
1993 10.163644193294468
1994 10.157314044784908
1995 10.150989415172146
1996 10.144670298270766
1997 10.138356687903817
1998 10.132048577902816
1999 10.12574596210772
2000 10.11944883436691
2001 10.113157188537196
2002 10.106871018483783
2003 10.100590318080267
2004 10.09431508120862
2005 10.088045301759179
2006 10.081780973630627
2007 10.075522090729985
2008 10.069268646972594
2

2967 5.903275958032146
2968 5.9003174347963
2969 5.897360987149556
2970 5.894406613219946
2971 5.891454311137559
2972 5.888504079034551
2973 5.885555915045131
2974 5.882609817305567
2975 5.8796657839541755
2976 5.876723813131326
2977 5.873783902979435
2978 5.87084605164296
2979 5.867910257268407
2980 5.864976518004314
2981 5.862044832001261
2982 5.859115197411857
2983 5.856187612390746
2984 5.853262075094601
2985 5.850338583682117
2986 5.847417136314016
2987 5.84449773115304
2988 5.841580366363948
2989 5.838665040113514
2990 5.8357517505705285
2991 5.832840495905788
2992 5.829931274292098
2993 5.82702408390427
2994 5.824118922919117
2995 5.821215789515452
2996 5.818314681874086
2997 5.815415598177824
2998 5.812518536611462
2999 5.809623495361789
3000 5.806730472617576
3001 5.803839466569582
3002 5.800950475410546
3003 5.798063497335187
3004 5.795178530540202
3005 5.792295573224259
3006 5.789414623588
3007 5.786535679834033
3008 5.783658740166938
3009 5.780783802793253
3010 5.7779108659

3967 3.738177703742339
3968 3.7366171864525963
3969 3.735057581108591
3970 3.733498887025201
3971 3.731941103517929
3972 3.730384229902908
3973 3.7288282654968987
3974 3.727273209617289
3975 3.7257190615820917
3976 3.724165820709947
3977 3.7226134863201183
3978 3.721062057732495
3979 3.719511534267589
3980 3.7179619152465344
3981 3.7164131999910905
3982 3.714865387823635
3983 3.7133184780671673
3984 3.7117724700453087
3985 3.7102273630822977
3986 3.7086831565029925
3987 3.7071398496328714
3988 3.7055974417980266
3989 3.7040559323251703
3990 3.7025153205416306
3991 3.7009756057753487
3992 3.6994367873548826
3993 3.697898864609405
3994 3.696361836868701
3995 3.6948257034631684
3996 3.6932904637238186
3997 3.691756116982274
3998 3.690222662570766
3999 3.6886900998221392
4000 3.6871584280698464
4001 3.6856276466479487
4002 3.6840977548911162
4003 3.682568752134628
4004 3.681040637714367
4005 3.6795134109668246
4006 3.6779870712290976
4007 3.6764616178388865
4008 3.674937050134499
4009 3.67

5164 2.370117838262292
5165 2.3692932776617317
5166 2.3684691186420577
5167 2.367645360951832
5168 2.366822004339809
5169 2.3659990485549356
5170 2.3651764933463504
5171 2.3643543384633836
5172 2.363532583655558
5173 2.3627112286725858
5174 2.3618902732643736
5175 2.361069717181017
5176 2.3602495601728033
5177 2.359429801990211
5178 2.3586104423839087
5179 2.357791481104755
5180 2.3569729179038
5181 2.3561547525322837
5182 2.355336984741635
5183 2.3545196142834746
5184 2.3537026409096105
5185 2.3528860643720417
5186 2.352069884422956
5187 2.351254100814731
5188 2.350438713299932
5189 2.3496237216313145
5190 2.348809125561821
5191 2.3479949248445835
5192 2.3471811192329226
5193 2.3463677084803467
5194 2.3455546923405506
5195 2.344742070567419
5196 2.343929842915024
5197 2.3431180091376236
5198 2.3423065689896645
5199 2.341495522225779
5200 2.340684868600788
5201 2.3398746078696986
5202 2.3390647397877036
5203 2.3382552641101837
5204 2.3374461805927043
5205 2.336637488991018
5206 2.33582

6111 1.739130436729399
6112 1.738595860451312
6113 1.7380615142053784
6114 1.7375273978643417
6115 1.7369935113010313
6116 1.7364598543883631
6117 1.735926426999339
6118 1.7353932290070466
6119 1.734860260284659
6120 1.7343275207054345
6121 1.7337950101427178
6122 1.7332627284699396
6123 1.7327306755606147
6124 1.732198851288345
6125 1.7316672555268162
6126 1.7311358881498002
6127 1.730604749031154
6128 1.730073838044819
6129 1.7295431550648226
6130 1.7290126999652762
6131 1.7284824726203782
6132 1.72795247290441
6133 1.7274227006917375
6134 1.7268931558568135
6135 1.7263638382741733
6136 1.7258347478184375
6137 1.7253058843643116
6138 1.724777247786586
6139 1.7242488379601337
6140 1.7237206547599138
6141 1.7231926980609693
6142 1.7226649677384267
6143 1.7221374636674973
6144 1.7216101857234767
6145 1.721083133781743
6146 1.7205563077177604
6147 1.7200297074070754
6148 1.7195033327253184
6149 1.7189771835482048
6150 1.7184512597515318
6151 1.7179255612111815
6152 1.71740008780312
6153 

7124 1.2967610108237204
7125 1.2964065660420296
7126 1.2960522568858917
7127 1.2956980832885876
7128 1.2953440451834386
7129 1.2949901425038055
7130 1.2946363751830898
7131 1.2942827431547328
7132 1.2939292463522154
7133 1.2935758847090588
7134 1.2932226581588246
7135 1.2928695666351133
7136 1.2925166100715657
7137 1.2921637884018633
7138 1.2918111015597262
7139 1.291458549478915
7140 1.2911061320932298
7141 1.2907538493365105
7142 1.290401701142637
7143 1.290049687445528
7144 1.2896978081791433
7145 1.289346063277481
7146 1.2889944526745793
7147 1.2886429763045164
7148 1.2882916341014092
7149 1.2879404259994147
7150 1.2875893519327293
7151 1.2872384118355888
7152 1.2868876056422685
7153 1.286536933287083
7154 1.2861863947043866
7155 1.2858359898285723
7156 1.285485718594073
7157 1.2851355809353604
7158 1.2847855767869463
7159 1.2844357060833809
7160 1.2840859687592538
7161 1.2837363647491942
7162 1.2833868939878699
7163 1.2830375564099883
7164 1.2826883519502956
7165 1.282339280543577

8205 0.9807354378437096
8206 0.9804957106615602
8207 0.9802560655125809
8208 0.9800165023606824
8209 0.9797770211697948
8210 0.9795376219038671
8211 0.9792983045268681
8212 0.9790590690027862
8213 0.9788199152956284
8214 0.978580843369422
8215 0.9783418531882128
8216 0.9781029447160661
8217 0.9778641179170671
8218 0.9776253727553194
8219 0.9773867091949464
8220 0.9771481272000911
8221 0.976909626734915
8222 0.976671207763599
8223 0.9764328702503439
8224 0.9761946141593689
8225 0.9759564394549127
8226 0.9757183461012335
8227 0.9754803340626084
8228 0.9752424033033336
8229 0.9750045537877245
8230 0.9747667854801159
8231 0.9745290983448615
8232 0.9742914923463339
8233 0.9740539674489254
8234 0.9738165236170468
8235 0.9735791608151287
8236 0.9733418790076199
8237 0.9731046781589888
8238 0.9728675582337228
8239 0.9726305191963283
8240 0.9723935610113307
8241 0.9721566836432743
8242 0.9719198870567227
8243 0.9716831712162585
8244 0.9714465360864825
8245 0.9712099816320156
8246 0.970973507817

9216 0.7748005542861348
9217 0.7746282045762598
9218 0.7744559085374116
9219 0.7742836661481024
9220 0.7741114773868552
9221 0.773939342232203
9222 0.7737672606626897
9223 0.7735952326568698
9224 0.773423258193308
9225 0.7732513372505798
9226 0.7730794698072706
9227 0.7729076558419766
9228 0.772735895333305
9229 0.7725641882598725
9230 0.7723925346003068
9231 0.772220934333246
9232 0.7720493874373386
9233 0.7718778938912437
9234 0.7717064536736304
9235 0.7715350667631787
9236 0.7713637331385786
9237 0.7711924527785309
9238 0.7710212256617465
9239 0.7708500517669467
9240 0.7706789310728636
9241 0.7705078635582392
9242 0.7703368492018262
9243 0.7701658879823875
9244 0.7699949798786963
9245 0.7698241248695364
9246 0.769653322933702
9247 0.7694825740499973
9248 0.769311878197237
9249 0.7691412353542463
9250 0.7689706454998605
9251 0.7688001086129255
9252 0.7686296246722972
9253 0.7684591936568422
9254 0.7682888155454369
9255 0.7681184903169688
9256 0.7679482179503345
9257 0.767777998424441

### 4.2 - Probability Distribution of Metalicity over the time:

In [11]:

Z_array    =np.array(Z_array)
m_gas_array=np.array(m_gas_array)
#pdf_array=[]
def pdf_metals(z,Mg,p=0.01):
    return (1 - np.exp( -1.*z/p))


n=100
z_range=np.linspace(0,Z_array[n],100)
#pdf_array=pdf_metals(Z_array,m_gas_array,p=0.01)
pdf_array=1.*pdf_metals(z_range,m_gas_array[n],p=0.01)/pdf_metals(Z_array[n],m_gas_array[n],p=0.01)
#1.*pdf_metals(z_range,m_gas_array[n],p=0.01)/
